In [1]:
import os
import fnmatch

import numpy as np
import pandas as pd

In [2]:
def find_target_filePath(dataPath):
    
    for folder in os.listdir(dataPath):
        for file in os.listdir(dataPath+'/'+folder):
            if fnmatch.fnmatch(file, '?_lvr_land_?.csv'):
                full_path = dataPath+'/'+folder+'/'+file
                yield full_path

In [3]:
def read_data(path):
    try:
        df = pd.read_csv(path, header=1) 
    except pd.errors.EmptyDataError:
        print('Note: filename.csv was empty. Skipping.')
        df = pd.DataFrame()
        
    return df

In [4]:
def preprocessing(df, path):
    # last one will be filename, second from the back will be that folder
    keyWord = path.split('/')
    
    # ./data/106_1/H_lvr_land_B.csv -> 106_1_H_B
    df_name = keyWord[-2]+'_'+keyWord[-1][0]+'_'+keyWord[-1][-5]
    df['df_name'] = df_name
    # floor: ch->digit
    df['total floor number'] = df['total floor number'].apply(ch2num)
    
    return df

In [5]:
number_map = {
    "零": 0,
    "一": 1,
    "二": 2,
    "三": 3,
    "四": 4,
    "五": 5,
    "六": 6,
    "七": 7,
    "八": 8,
    "九": 9,
    "十":10
}

def ch2num(ch_num):
    num_list = []
    num = 0
    
#     print(ch_num)
    
    if ch_num == '地下層':
        return -1
    
    try:
        ch_num = int(ch_num)
    except:
        pass
    
    if ch_num is np.nan:
        return 0
    if isinstance(ch_num, int):
        return ch_num
    
    for ch in ch_num:
        if ch == '層':
            continue
        num_list.append(number_map[ch])
        
    if len(num_list) == 1:
        return num_list[0]
    elif len(num_list) == 2:
        # 10,20,30,40...,90
        if num_list[1] == 10:
            return num_list[0]*10
        else:
            # 11-19
            return num_list[0]+num_list[1]
    else:
        return num_list[0]*10+num_list[2]

In [6]:
# step 1
# dataPath = './testDataSet'
dataPath = './data'

number_rows = []
target_df = []

# step 2   
for filePath in find_target_filePath(dataPath):
    print(filePath)
    
    df = read_data(filePath)
    if df.empty:
        continue
    
    df_after = preprocessing(df, filePath)
    number_rows.append(len(df_after.index))
    target_df.append(df_after)
    
# step 3    
df_all = pd.concat(target_df)

./data/106_1/A_lvr_land_A.csv
./data/106_1/B_lvr_land_B.csv
./data/106_1/E_lvr_land_A.csv
./data/106_1/F_lvr_land_A.csv
./data/106_1/H_lvr_land_B.csv
./data/106_2/A_lvr_land_A.csv
./data/106_2/B_lvr_land_B.csv
Note: filename.csv was empty. Skipping.
./data/106_2/E_lvr_land_A.csv
./data/106_2/F_lvr_land_A.csv
./data/106_2/H_lvr_land_B.csv
./data/106_3/A_lvr_land_A.csv
./data/106_3/B_lvr_land_B.csv
./data/106_3/E_lvr_land_A.csv
./data/106_3/F_lvr_land_A.csv
./data/106_3/H_lvr_land_B.csv
./data/106_4/A_lvr_land_A.csv
./data/106_4/B_lvr_land_B.csv
./data/106_4/E_lvr_land_A.csv
./data/106_4/F_lvr_land_A.csv
./data/106_4/H_lvr_land_B.csv
./data/107_1/A_lvr_land_A.csv
./data/107_1/B_lvr_land_B.csv
./data/107_1/E_lvr_land_A.csv
./data/107_1/F_lvr_land_A.csv
./data/107_1/H_lvr_land_B.csv
./data/107_2/A_lvr_land_A.csv
./data/107_2/B_lvr_land_B.csv
./data/107_2/E_lvr_land_A.csv
./data/107_2/F_lvr_land_A.csv
./data/107_2/H_lvr_land_B.csv
./data/107_3/A_lvr_land_A.csv
./data/107_3/B_lvr_land_B.csv


In [19]:
# df_all

In [8]:
# filter condition:
# 主要用途: main use -> chinese (住家用)
# 建物型態: building state -> chinese (住宅大樓(11層含以上有電梯))
# 總樓層數: total floor number -> chinese with unit (>= 十三層)

In [9]:
filter_result = df_all[(df_all['main use']=='住家用')&
                       (df_all['building state']=='住宅大樓(11層含以上有電梯)')&
                       (df_all['total floor number']>=13)]

In [10]:
# filter_result[['main use','building state','total floor number']]

In [20]:
filter_result.to_csv('./filter.csv', encoding='utf_8_sig')

In [12]:
# count condition:
# 總件數 count(*)
# 總車位數 count('berth shifting total area square meter' != 0)
# 平均總價元 AVG('total price NTD')
# 平均車位總價元 AVG('the berth total price NTD')

In [13]:
total_count = len(df_all.index)

In [14]:
total_number_parking = len(df_all[(df_all['berth shifting total area square meter'] > 0)])

In [15]:
avg_total_price = df_all['total price NTD'].mean()

In [16]:
avg_total_parking_price = df_all['the berth total price NTD'].mean()

In [21]:
output_dict = {'total_count':total_count,
               'total_number_parking':total_number_parking,
               'avg_total_price':avg_total_price,
               'avg_total_parking_price':avg_total_parking_price}
data = pd.DataFrame(output_dict, index=[0])
data.to_csv('./count.csv', encoding='utf_8_sig')